In [2]:
!unzip "/content/drive/MyDrive/New folder.zip" -d "/content/flowers"


Streaming output truncated to the last 5000 lines.
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07798.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07799.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07800.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07801.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07802.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07803.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07804.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07805.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07806.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07807.jpg  
  inflating: /content/flowers/flower102/new_train/mexican_petunia/image_07808.jpg  
  inflating: /content/flo

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from torch.utils.data import DataLoader
from google.colab import drive
from PIL import Image

def mount_drive():
    drive.mount('/content/drive')
    dataset_path = "/content/flowers/flower17"  # Cập nhật đường dẫn tới dataset
    return dataset_path

def prepare_dataloaders(dataset_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
    val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'validation'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_dataset.classes), train_dataset.classes

def create_alexnet_model(num_classes):
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer cho số lớp
    return model

def evaluate_model(model, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

    evaluate_model(model, val_loader, device)
    print("Training complete")

def predict_image(model, image_path, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(image.to(device))
        _, predicted = output.max(1)

    predicted_class = class_names[predicted.item()]
    print(f"Predicted Flower: {predicted_class}")
    return predicted_class

def main():
    dataset_path = mount_drive()
    train_loader, val_loader, num_classes, class_names = prepare_dataloaders(dataset_path)
    model = create_alexnet_model(num_classes)
    train_model(model, train_loader, val_loader)

    test_image_path = "/content/flowers/flower17/train/bluebell/image_0251.jpg"  # Cập nhật đường dẫn ảnh test
    predict_image(model, test_image_path, class_names)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 2.9331, Accuracy: 9.64%
Epoch 2/10, Loss: 2.8353, Accuracy: 4.68%
Epoch 3/10, Loss: 2.8321, Accuracy: 5.99%
Epoch 4/10, Loss: 2.8237, Accuracy: 5.62%
Epoch 5/10, Loss: 2.8202, Accuracy: 5.99%
Epoch 6/10, Loss: 2.8218, Accuracy: 5.81%
Epoch 7/10, Loss: 2.8231, Accuracy: 5.81%
Epoch 8/10, Loss: 2.8214, Accuracy: 5.43%
Epoch 9/10, Loss: 2.8209, Accuracy: 6.09%
Epoch 10/10, Loss: 2.8198, Accuracy: 6.55%
Validation Accuracy: 5.88%
Training complete
Predicted Flower: lily_valley


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from torch.utils.data import DataLoader
from google.colab import drive
from PIL import Image

def mount_drive():
    drive.mount('/content/drive')
    dataset_path = "/content/flowers/flower17"  # Cập nhật đường dẫn tới dataset
    return dataset_path

def prepare_dataloaders(dataset_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
    val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'validation'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_dataset.classes), train_dataset.classes

def create_vgg16_model(num_classes):
    model = models.vgg16(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer cho số lớp
    return model

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

    print("Training complete")

def evaluate_model(model, val_loader, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

    return accuracy

def predict_image(model, image_path, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = output.max(1)

    predicted_class = class_names[predicted.item()]
    print(f"Predicted flower: {predicted_class}")
    return predicted_class

def main():
    dataset_path = mount_drive()
    train_loader, val_loader, num_classes, class_names = prepare_dataloaders(dataset_path)
    model = create_vgg16_model(num_classes)
    train_model(model, train_loader, val_loader)
    evaluate_model(model, val_loader, class_names)

    # Dự đoán một ảnh mẫu
    sample_image_path = "/content/flowers/flower17/train/bluebell/image_0251.jpg"  # Cập nhật đường dẫn ảnh mẫu
    predict_image(model, sample_image_path, class_names)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s] 


Epoch 1/10, Loss: 3.0827, Accuracy: 6.27%
Epoch 2/10, Loss: 2.8566, Accuracy: 6.27%
Epoch 3/10, Loss: 2.7804, Accuracy: 9.46%
Epoch 4/10, Loss: 2.5906, Accuracy: 11.52%
Epoch 5/10, Loss: 2.4705, Accuracy: 13.95%
Epoch 6/10, Loss: 2.3361, Accuracy: 20.69%
Epoch 7/10, Loss: 2.0985, Accuracy: 28.00%
Epoch 8/10, Loss: 2.0041, Accuracy: 29.31%
Epoch 9/10, Loss: 1.8992, Accuracy: 34.93%
Epoch 10/10, Loss: 1.7529, Accuracy: 40.92%
Training complete
Validation Accuracy: 34.71%
Predicted flower: crocus


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from torch.utils.data import DataLoader
from google.colab import drive
from PIL import Image

def mount_drive():
    drive.mount('/content/drive')
    dataset_path = "/content/flowers/flower17"  # Cập nhật đường dẫn tới dataset
    return dataset_path

def prepare_dataloaders(dataset_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
    val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'validation'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_dataset.classes), train_dataset.classes

def create_alexnet_model(num_classes):
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer cho số lớp
    return model

def create_vgg16_model(num_classes):
    model = models.vgg16(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer
    return model

def evaluate_model(model, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

def train_model(model, train_loader, val_loader, model_name, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"Training {model_name} model...")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        print(f"{model_name} - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

    evaluate_model(model, val_loader, device)
    print(f"{model_name} training complete")

def predict_image(model, image_path, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(image.to(device))
        _, predicted = output.max(1)

    predicted_class = class_names[predicted.item()]
    print(f"Predicted Flower: {predicted_class}")
    return predicted_class

def main():
    dataset_path = mount_drive()
    train_loader, val_loader, num_classes, class_names = prepare_dataloaders(dataset_path)

    # Train and evaluate AlexNet
    alexnet_model = create_alexnet_model(num_classes)
    train_model(alexnet_model, train_loader, val_loader, "AlexNet")

    # Train and evaluate VGG16
    vgg16_model = create_vgg16_model(num_classes)
    train_model(vgg16_model, train_loader, val_loader, "VGG16")

    # Predict using the trained VGG16 model
    test_image_path =  "/content/flowers/flower17/train/bluebell/image_0251.jpg"  # Cập nhật đường dẫn ảnh test
    predict_image(vgg16_model, test_image_path, class_names)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training AlexNet model...
AlexNet - Epoch 1/10, Loss: 2.6298, Accuracy: 18.73%
AlexNet - Epoch 2/10, Loss: 2.6733, Accuracy: 14.79%
AlexNet - Epoch 3/10, Loss: 2.5674, Accuracy: 16.10%
AlexNet - Epoch 4/10, Loss: 2.3470, Accuracy: 21.54%
AlexNet - Epoch 5/10, Loss: 2.2052, Accuracy: 26.69%
AlexNet - Epoch 6/10, Loss: 2.0020, Accuracy: 29.87%
AlexNet - Epoch 7/10, Loss: 1.9156, Accuracy: 32.30%
AlexNet - Epoch 8/10, Loss: 1.8474, Accuracy: 35.02%
AlexNet - Epoch 9/10, Loss: 1.7304, Accuracy: 36.70%
AlexNet - Epoch 10/10, Loss: 1.7748, Accuracy: 36.89%
Validation Accuracy: 43.53%
AlexNet training complete


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training VGG16 model...
VGG16 - Epoch 1/10, Loss: 2.8982, Accuracy: 6.18%
VGG16 - Epoch 2/10, Loss: 2.8165, Accuracy: 7.12%
VGG16 - Epoch 3/10, Loss: 2.6484, Accuracy: 13.11%
VGG16 - Epoch 4/10, Loss: 2.5947, Accuracy: 10.49%
VGG16 - Epoch 5/10, Loss: 2.4774, Accuracy: 14.51%
VGG16 - Epoch 6/10, Loss: 2.4117, Accuracy: 15.64%
VGG16 - Epoch 7/10, Loss: 2.2896, Accuracy: 20.51%
VGG16 - Epoch 8/10, Loss: 2.2911, Accuracy: 22.10%
VGG16 - Epoch 9/10, Loss: 2.3433, Accuracy: 22.19%
VGG16 - Epoch 10/10, Loss: 2.2541, Accuracy: 24.16%
Validation Accuracy: 21.76%
VGG16 training complete
Predicted Flower: tigerlily


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import timm
import os
from torch.utils.data import DataLoader
from google.colab import drive
from PIL import Image

def mount_drive():
    drive.mount('/content/drive')
    dataset_path = "/content/flowers/flower17"  # Cập nhật đường dẫn tới dataset
    return dataset_path

def prepare_dataloaders(dataset_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
    val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'validation'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_dataset.classes), train_dataset.classes

def create_alexnet_model(num_classes):
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer cho số lớp
    return model

def create_vgg16_model(num_classes):
    model = models.vgg16(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)  # Chỉnh sửa output layer
    return model

def create_swin_model(num_classes):
    model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=num_classes)
    return model

def evaluate_model(model, val_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

def train_model(model, train_loader, val_loader, model_name, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"Training {model_name} model...")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        print(f"{model_name} - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

    evaluate_model(model, val_loader, device)
    print(f"{model_name} training complete")

def predict_image(model, image_path, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(image.to(device))
        _, predicted = output.max(1)

    predicted_class = class_names[predicted.item()]
    print(f"Predicted Flower: {predicted_class}")
    return predicted_class

def main():
    dataset_path = mount_drive()
    train_loader, val_loader, num_classes, class_names = prepare_dataloaders(dataset_path)

    # Train and evaluate AlexNet
    alexnet_model = create_alexnet_model(num_classes)
    train_model(alexnet_model, train_loader, val_loader, "AlexNet")

    # Train and evaluate VGG16
    vgg16_model = create_vgg16_model(num_classes)
    train_model(vgg16_model, train_loader, val_loader, "VGG16")

    # Train and evaluate SWIN Transformer
    swin_model = create_swin_model(num_classes)
    train_model(swin_model, train_loader, val_loader, "SWIN Transformer")

    # Predict using the trained SWIN model
    test_image_path = "/content/flowers/flower17/train/bluebell/image_0251.jpg"  # Cập nhật đường dẫn ảnh test
    predict_image(swin_model, test_image_path, class_names)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training AlexNet model...
AlexNet - Epoch 1/10, Loss: 2.7722, Accuracy: 15.73%
AlexNet - Epoch 2/10, Loss: 2.7664, Accuracy: 10.58%
AlexNet - Epoch 3/10, Loss: 2.7617, Accuracy: 7.58%
AlexNet - Epoch 4/10, Loss: 2.7708, Accuracy: 8.05%
AlexNet - Epoch 5/10, Loss: 2.8587, Accuracy: 5.71%
AlexNet - Epoch 6/10, Loss: 2.8204, Accuracy: 5.90%
AlexNet - Epoch 7/10, Loss: 2.8207, Accuracy: 5.34%
AlexNet - Epoch 8/10, Loss: 2.8212, Accuracy: 5.06%
AlexNet - Epoch 9/10, Loss: 2.8198, Accuracy: 5.24%
AlexNet - Epoch 10/10, Loss: 2.8195, Accuracy: 5.90%
Validation Accuracy: 5.88%
AlexNet training complete


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training VGG16 model...
VGG16 - Epoch 1/10, Loss: 2.9682, Accuracy: 5.52%
VGG16 - Epoch 2/10, Loss: 2.8264, Accuracy: 7.96%
VGG16 - Epoch 3/10, Loss: 2.7628, Accuracy: 8.71%
VGG16 - Epoch 4/10, Loss: 2.6272, Accuracy: 15.17%
VGG16 - Epoch 5/10, Loss: 2.5357, Accuracy: 17.23%
VGG16 - Epoch 6/10, Loss: 2.3954, Accuracy: 17.98%
VGG16 - Epoch 7/10, Loss: 2.1452, Accuracy: 27.34%
VGG16 - Epoch 8/10, Loss: 2.0267, Accuracy: 30.62%
VGG16 - Epoch 9/10, Loss: 1.8483, Accuracy: 37.83%
VGG16 - Epoch 10/10, Loss: 1.6270, Accuracy: 45.13%
Validation Accuracy: 41.76%
VGG16 training complete


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Training SWIN Transformer model...
SWIN Transformer - Epoch 1/10, Loss: 2.3858, Accuracy: 23.22%
SWIN Transformer - Epoch 2/10, Loss: 1.9747, Accuracy: 32.68%
SWIN Transformer - Epoch 3/10, Loss: 1.7520, Accuracy: 37.36%
SWIN Transformer - Epoch 4/10, Loss: 1.4818, Accuracy: 49.06%
SWIN Transformer - Epoch 5/10, Loss: 1.3113, Accuracy: 54.12%
SWIN Transformer - Epoch 6/10, Loss: 0.9960, Accuracy: 64.51%
SWIN Transformer - Epoch 7/10, Loss: 0.8797, Accuracy: 70.22%
SWIN Transformer - Epoch 8/10, Loss: 0.7455, Accuracy: 75.37%
SWIN Transformer - Epoch 9/10, Loss: 0.5661, Accuracy: 81.09%
SWIN Transformer - Epoch 10/10, Loss: 0.5070, Accuracy: 83.61%
Validation Accuracy: 72.35%
SWIN Transformer training complete
Predicted Flower: bluebell


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
import numpy as np
from torch.utils.data import DataLoader
from google.colab import drive
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier

def mount_drive():
    drive.mount('/content/drive')
    dataset_path = "/content/flowers/flower17"  # Cập nhật đường dẫn tới dataset
    return dataset_path

def prepare_dataloaders(dataset_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
    val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, 'validation'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, len(train_dataset.classes), train_dataset.classes, train_dataset

def create_alexnet_model(num_classes):
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def create_vgg16_model(num_classes):
    model = models.vgg16(pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def extract_features(model, dataloader, device):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, label in dataloader:
            images = images.to(device)
            output = model.features(images)  # Trích xuất đặc trưng
            output = torch.flatten(output, start_dim=1).cpu().numpy()
            features.append(output)
            labels.append(label.numpy())
    return np.vstack(features), np.hstack(labels)

def train_knn(train_loader, model, device, k=7):
    features, labels = extract_features(model, train_loader, device)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(features, labels)
    return knn

def predict_with_knn(knn, model, image_path, device, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        feature = model.features(image)
        feature = torch.flatten(feature, start_dim=1).cpu().numpy()

    predicted_label = knn.predict(feature)[0]
    print(f"Predicted Flower (KNN): {class_names[predicted_label]}")
    return class_names[predicted_label]

def evaluate_knn(knn, model, dataloader, device):
    features, labels = extract_features(model, dataloader, device)
    predictions = knn.predict(features)
    accuracy = (predictions == labels).mean() * 100
    print(f"KNN Model Accuracy: {accuracy:.2f}%")
    return accuracy

def main():
    dataset_path = mount_drive()
    train_loader, val_loader, num_classes, class_names, train_dataset = prepare_dataloaders(dataset_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train AlexNet
    alexnet_model = create_alexnet_model(num_classes).to(device)
    knn_alexnet = train_knn(train_loader, alexnet_model, device, k=7)

    # Train VGG16
    vgg16_model = create_vgg16_model(num_classes).to(device)
    knn_vgg16 = train_knn(train_loader, vgg16_model, device, k=7)

    # Predict with KNN (AlexNet features)
    test_image_path = "/content/flowers/flower17/train/bluebell/image_0251.jpg"  # Cập nhật đường dẫn ảnh test
    evaluate_knn(knn_alexnet, alexnet_model, val_loader, device)
    predict_with_knn(knn_alexnet, alexnet_model, test_image_path, device, class_names)

    # Predict with KNN (VGG16 features)
    evaluate_knn(knn_vgg16, vgg16_model, val_loader, device)
    predict_with_knn(knn_vgg16, vgg16_model, test_image_path, device, class_names)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KNN Model Accuracy: 71.18%
Predicted Flower (KNN): bluebell
KNN Model Accuracy: 60.00%
Predicted Flower (KNN): bluebell
